In [ ]:
###### Library de base
import pandas as pd
import datetime
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import gc

###### Librabary Sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor

###### Library LightGBM



import lightgbm as lgb

# Importation de la base

In [ ]:
##############
#### Importation de toute la base
##############


train=pd.read_csv("/kaggle/input/ashrae-energy-prediction/train.csv",sep=",")
weather_train=pd.read_csv("/kaggle/input/ashrae-energy-prediction/weather_train.csv",sep=",")
building_meta=pd.read_csv("/kaggle/input/ashrae-energy-prediction/building_metadata.csv",sep=',')

# Definition de fonction


In [ ]:

##############
#### Fonction pour réduire la taille de la base
##############


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Source : https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction


##############
#### Fonction pour calculer les données manquantes
##############

def val_manq(df):
    percent = (df.isnull().sum()).sort_values(ascending=False)
    percent.plot(kind='bar', figsize = (20,10), fontsize=20)
    plt.xlabel('Colonnes')
    plt.ylabel('Nombre de lignes')
    plt.title('Tot valeur manquante')


In [ ]:

##############
#### Visualisation des valeurs manquantes
##############


val_manq(weather_train)
plt.show()
val_manq(train)
plt.show()
val_manq(building_meta)


In [ ]:
# Remplissage des données manquantes

print("Date du début de l'étude : "+ min(train["timestamp"]))
print("Date de la fin de l'étude : "+max(train["timestamp"]))



In [ ]:
##############
#### En comparant le nombre d'heure en 2016 et le nombre d'heure dans 
#### la base weather on comprend bien qu'il y a un pb.
##############


print("Nombre d'heures dans l'année 2016 : " + str(366*24)) 
print("Nombre d'heures pour chaque site dans l'année 2016 : " + str(366*24*16)) 
print("Nombre d'heures dans la base weather : " + str(len(weather_train["timestamp"])))

Normalement la base de données weather_train à autant de données que d'heures dans l'années 2016 multiplié par le nombre de lieu où des mesures ont été effectués, c'est à dire le nombre de site_id différents. Cependant en faisant les calculs nous nous sommes rendus compte qu'il manquait environ 1000 heures mesurés, 139773 lignes dans la base weather_train alors qu'il devrait en avoir 140544. Nous avons décidés de rajouter ses lignes manquantes.


In [ ]:
##############
#### Problème de date manquante
##############

format_date = "%Y-%m-%d %H:%M:%S"

date_deb = min(weather_train["timestamp"])
date_fin = max(weather_train["timestamp"])

diff = 366*24 # Nombres d'heures pendant toute la durée de l'étude

début = datetime.datetime.strptime(date_deb,format_date)
# date time de la valeur de départ de l'étude
fin = datetime.datetime.strptime(date_fin,format_date)
# date time de la valeur de fin de l'étude

date_list = [(début + datetime.timedelta(hours=x)).strftime(format_date) for x in range(diff)]
# Création d'une liste de date sous le format timestamp

for i in range(0,16):
    heure_site = weather_train[weather_train["site_id"]==i]["timestamp"]
    heure_site = heure_site.values.tolist()
    heure_manquante = list(set(date_list)-set(heure_site))
    rangs_manquants = pd.DataFrame({'timestamp' : heure_manquante,'site_id' : i})
    weather_train = pd.concat([rangs_manquants,weather_train],sort=False)
# Créations des lignes manquantes de la base

In [ ]:
##############
#### Création de nouvelles colonnes
##############

weather_train["datetime"] = pd.to_datetime(weather_train["timestamp"])
weather_train["annee"] = weather_train["datetime"].dt.year
weather_train["mois"] = weather_train["datetime"].dt.month
weather_train["semaine"] = weather_train["datetime"].dt.week
weather_train["jour"] = weather_train["datetime"].dt.day
weather_train["heure"] = weather_train["datetime"].dt.hour
weather_train.drop(columns=["datetime"],inplace=True)

In [ ]:
##############
#### Remplisssage des données manquantes de la base weather par des moyennes
##############


col_weather=["air_temperature","cloud_coverage","dew_temperature","precip_depth_1_hr","sea_level_pressure","wind_direction","wind_speed"]
mean_weather=weather_train.groupby(["site_id","mois"]).transform(lambda x: x.fillna(x.mean()))
for col in col_weather:
    weather_train[col] = mean_weather[col]
weather_train=weather_train.fillna(method='ffill')


Comme nous avons pu le voir nos bases possèdent de nombreuses données manquantes que se soit dans la base weather ou dans la base building_metadata.  
Pour la base weather nous remplancer les données manquantes par des approximations (i.e des moyennes) des valeurs du mois si celà est possible est sinon nous allons les remplacer par la valeur la plus proche non nulles dans la base.



# Manipulation de la base

In [ ]:
##############
#### Encoding d'une variables qualitative
##############
#A faire après les graphiques

labelencoder=LabelEncoder()
building_meta["primary_use"] = labelencoder.fit_transform(building_meta["primary_use"])

In [ ]:

##############
#### Merge des 3 bases
##############



df=pd.merge(train,building_meta, left_on='building_id', right_on='building_id')
df=pd.merge(df,weather_train, on=['timestamp','site_id'])
df = reduce_mem_usage(df)


In [ ]:
##############
#### Conversion d'unité d'un certain type de meter à un certain endroit
##############

df.loc[(df["site_id"] == 0) & (df["meter"] == 0), "meter_reading"]=df.loc[(df["site_id"] == 0) & (df["meter"] == 0), "meter_reading"]*0.2931


# Data Vizualisation

In [ ]:
#matrice de correlation
corr= df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize = (10,10))
ax = sns.heatmap(corr.round(2), mask=mask,
            vmin=-1,     
            cmap='coolwarm',
            annot=True)

In [ ]:
primary_use=building_meta["primary_use"].value_counts().reset_index()
primary_use_perc=primary_use["primary_use"]*100/sum(primary_use["primary_use"])
nom=['Education','Office','Entertainment/public assembly' ,'Public services','Lodging/residential' ,'Other' ,'Healthcare','Parking','Warehouse/storage','Manufacturing/industrial','Retail','Services','Technology/science','Food sales and service','Utility','Religious worship']
y_position=range(0,len(primary_use))


plt.figure(figsize=(10,10))
A=plt.barh(y_position,primary_use_perc,
       color='#66b3ff', align="center")
plt.yticks(y_position,nom)

for k in range(len(primary_use_perc)):
    plt.text(primary_use_perc[k]+0.5,
             y_position[k]-0.15, 
             str(round(primary_use_perc[k],2))+'%', 
             fontsize=15,
             color='dimgrey')
    

plt.xlim([0, max(primary_use_perc)+10])
# Enléve l'échelle de l'axe x
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.xlabel("Pourcentage des batiments")
plt.ylabel('Type de chauffage')
plt.show()


In [ ]:
##############
#### Calcul des consommations moyennes de chaques meter type par semaine
##############

mediane_conso_semaine = df.groupby(['semaine']).median()
mediane_conso_semaine_0 = df[df["meter"]==0].groupby(['semaine']).median()
mediane_conso_semaine_1 = df[df["meter"]==1].groupby(['semaine']).median()
mediane_conso_semaine_2 = df[df["meter"]==2].groupby(['semaine']).median()
mediane_conso_semaine_3 = df[df["meter"]==3].groupby(['semaine']).median()

##############
#### Plot Standard
##############


plt.figure(figsize=(10,10))
plt.plot(range(0,len(mediane_conso_semaine)),mediane_conso_semaine["meter_reading"],label='mediane')
plt.plot(range(0,len(mediane_conso_semaine)),mediane_conso_semaine_0["meter_reading"],label='electricity')
plt.plot(range(0,len(mediane_conso_semaine)),mediane_conso_semaine_1["meter_reading"],label='chilledwater')
plt.plot(range(0,len(mediane_conso_semaine)),mediane_conso_semaine_2["meter_reading"],label='steam')
plt.plot(range(0,len(mediane_conso_semaine)),mediane_conso_semaine_3["meter_reading"],label='hotwater')
plt.ylabel('Consomation en Kwh')
plt.xlabel('Semaine')
plt.legend()
#plt.savefig('mediane_conso-semaine_primary_use.png')

In [ ]:
##############
#### Plot Log de la valeur
##############



plt.figure(figsize=(10,10))
plt.plot(range(0,len(mediane_conso_semaine)),np.log(mediane_conso_semaine["meter_reading"]),label='mediane')
plt.plot(range(0,len(mediane_conso_semaine)),np.log(mediane_conso_semaine_0["meter_reading"]),label='electricity')
plt.plot(range(0,len(mediane_conso_semaine)),np.log(mediane_conso_semaine_1["meter_reading"]),label='chilledwater')
plt.plot(range(0,len(mediane_conso_semaine)),np.log(mediane_conso_semaine_2["meter_reading"]),label='steam')
plt.plot(range(0,len(mediane_conso_semaine)),np.log(mediane_conso_semaine_3["meter_reading"]),label='hotwater')
plt.ylabel('log de la Consomation en Kwh')
plt.xlabel('Semaine')
plt.legend()
plt.savefig('semaine_conso.png')

del mediane_conso_semaine
del mediane_conso_semaine_0
del mediane_conso_semaine_1
del mediane_conso_semaine_2
del mediane_conso_semaine_3

In [ ]:
############
### meter_reading avant log
############
plt.figure(figsize=(10,10))
plt.hist(train["meter_reading"],bins=30,color='green')
plt.ylabel('meter_reading count')
plt.xlabel('value')
#plt.savefig('meter_reading_no_log.png')

In [ ]:
############
### meter_reading après log
############
plt.figure(figsize=(10,10))
plt.hist(np.log1p(train["meter_reading"]),bins=30,color='green')
plt.ylabel('meter_reading count')
plt.xlabel('value')
#plt.savefig('meter_reading_log.png')

Analyse du site 13 :

In [ ]:
##############
#### Représentation de "indicator of the primary category of activities of the 
#### building"
##############

fig, ax = plt.subplots(figsize=(10,8))
sns.countplot(y='meter', data=df[df['site_id']==13])
#plt.savefig('primary_use_build.png')

In [ ]:
##############
#### Représentation de "indicator of the primary category of activities of the 
#### building"
##############

fig, ax = plt.subplots(figsize=(10,8))
sns.countplot(y='primary_use', data=df[df['site_id']==13])
#plt.savefig('primary_use_build.png')

In [ ]:
df[df['site_id'] == 13].describe()

Consommation d'énergie en fonction des catégories d'utilisation des bâtiments et du type d'énergie 

In [ ]:

temp_df = df.groupby("primary_use").meter_reading.sum().sort_values()

plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"Primary use")
plt.tight_layout()
plt.show()

In [ ]:

temp_df = df.groupby("meter").meter_reading.sum().sort_values()

plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"Meter")
plt.tight_layout()
plt.show()


Analyse de la consommation d'énergie par site en total et par médiane

In [ ]:

temp_df = df.groupby("site_id").meter_reading.sum().sort_values()

plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"site_id")
plt.tight_layout()
plt.show()


In [ ]:
temp_df = df.groupby("site_id").meter_reading.median().sort_values()

plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"site_id")
plt.tight_layout()
plt.show()

Représentation de la consomamtion des bâtiments par année de construction

In [ ]:
df['year_built_bin'] = pd.cut(x=df['year_built'], bins=[1955, 1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020])
temp_df = df.groupby("year_built_bin").meter_reading.mean().sort_values()


plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"year_built")
plt.tight_layout()
plt.savefig('building_built_moy_5grp.png')
plt.show()

In [ ]:
temp_df = df.groupby("year_built_bin").meter_reading.median().sort_values()


plt.figure(figsize=(16,9))
temp_df.plot(kind="barh")
plt.xlabel("Sum of readings")
plt.ylabel(f"year_built")
plt.tight_layout()
plt.savefig('building_built__med_5grp.png')
plt.show()


# Split des données en Apprentissage/Test

In [ ]:
##############
#### Suppression des bases initiales
##############

del train
del weather_train
del building_meta

In [ ]:
##############
#### Création de la variable cible et des variables explicatioves
#### Split en App/Test
##############

df.drop(columns=["year_built","floor_count"],inplace=True)


X=df.drop(columns=["meter_reading","timestamp","year_built_bin"])
y=np.log1p(df["meter_reading"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


del X
del y
del df

gc.collect();

# Modèle de Prédiction



## KNN

### Modèle de base

In [ ]:
knn = KNeighborsRegressor(n_jobs=-1, weights='distance') # K = 5 par défauts
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)


### Recherche du meilleur modèle

In [ ]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
k_range = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
erreur_knn = []
for i in range(0,len(k_range)):
    knn = KNeighborsRegressor(n_neighbors=k_range[i],n_jobs=-1, weights='distance')
    knn.fit(X_train_knn,y_train_knn)
    y_pred = knn.predict(X_test_knn)
    erreur = mean_squared_error(y_test_knn, y_pred)**0.5
    erreur_knn.append(erreur)

del X_train_knn
del X_test_knn
del y_test_knn
del y_train_knn
del y_pred

gc.collect();

In [ ]:
erreur_knn

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(k_range,erreur_knn , marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)
plt.xlabel("Valeur de K")
plt.ylabel("Erreur du modèle")
plt.xticks(k_range,k_range);

#### Résultat de la recherche
Valeur = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]  :  
-Résultat = [1.660315121583663, 1.5799791843467181, 1.5444780998289551, 1.5241509842625436,1.5111512882369214, 1.5021696018398543,1.495729114354016, 1.4910569465040882, 1.4874995481154836, 1.4847853887052653, 1.4826815606432966, 1.481047281033797,1.4797644928403906, 1.4787621572620613]


### Test du meilleur modèle

In [ ]:
knn_best = KNeighborsRegressor(n_jobs=-1, weights='distance', n_neighbors=55) 
knn_best.fit(X_train,y_train)
y_pred = knn_best.predict(X_test)
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

## Random Forest

### Paramètre nécessaire

In [ ]:
##############
#### Création de forets dans le but de réduire la demande mémoire
##############


from sklearn.ensemble import forest
def set_rf_samples(n):
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))
set_rf_samples(130000)


def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

### Random Forest de base

In [ ]:
model = RandomForestRegressor(random_state=0,
                              n_jobs=-1); # n_estimators = 10
model.fit(X_train,y_train);

y_pred=model.predict(X_test);
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
##############
#### Importance des variables dans la foret d'arbre
##############

feature_names = X_train.columns
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,10))
plt.xlabel("Importance des Variables")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", align="center")
plt.yticks(range(X_train.shape[1]), feature_names)
#plt.xlim([-1, X_train.shape[1]])
plt.show()

### Recherche du meilleur modèle

In [ ]:
##############
#### Choix des paramètres 
##############


X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Paramètre

n_estimators =[10, 30, 50, 70, 100, 130, 160, 190, 220]
erreur_rf = []
for i in range(0,len(n_estimators)):
    rf = RandomForestRegressor(n_estimators=n_estimators[i],random_state=0,n_jobs=-1)
    rf.fit(X_train_rf,y_train_rf)
    y_pred = rf.predict(X_test_rf)
    erreur = mean_squared_error(y_test_rf, y_pred)**0.5
    erreur_rf.append(erreur)

del X_train_rf
del X_test_rf
del y_train_rf
del y_test_rf

gc.collect();

In [ ]:
print(erreur_rf)

In [ ]:
erreur_n_estimators = erreur_rf
plt.figure(figsize=(10,10))
plt.plot(n_estimators,erreur_n_estimators, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4);
plt.xlabel("Nombre d'arbres")
plt.ylabel("Erreur")
plt.xticks(n_estimators,n_estimators);

In [ ]:
##############
#### Choix des paramètres 
##############


X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Paramètre
max_depth = [10,20, 30, 40, 50, 60, 70, 80, 90, 100, None]
erreur_rf = []
for i in range(0,len(max_depth)):
    rf = RandomForestRegressor(max_depth=max_depth[i],random_state=0,n_jobs=-1 )
    rf.fit(X_train_rf,y_train_rf)
    y_pred = rf.predict(X_test_rf)
    erreur = mean_squared_error(y_test_rf, y_pred)**0.5
    erreur_rf.append(erreur)

del X_train_rf
del X_test_rf
del y_train_rf
del y_test_rf

gc.collect();

In [ ]:
print(erreur_rf)

In [ ]:
erreur_max_depth = erreur_rf
plt.figure(figsize=(10,10))
plt.plot(max_depth,erreur_max_depth, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4);
plt.xlabel("Profondeur maximale des arbres")
plt.ylabel("Erreur")
plt.xticks(max_depth[:-1],max_depth[:-1]); # Car le None ne peut pas être affiché

In [ ]:
##############
#### Choix des paramètres 
##############


X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Paramètre
min_samples_split = [2, 5, 10, 20 ,30]
erreur_rf = []
for i in range(0,len(min_samples_split)):
    rf = RandomForestRegressor(min_samples_split=min_samples_split[i],random_state=0,n_jobs=-1 )
    rf.fit(X_train_rf,y_train_rf)
    y_pred = rf.predict(X_test_rf)
    erreur = mean_squared_error(y_test_rf, y_pred)**0.5
    erreur_rf.append(erreur)

del X_train_rf
del X_test_rf
del y_train_rf
del y_test_rf

gc.collect();


In [ ]:
print(erreur_rf)

In [ ]:
erreur_min_samples_split = erreur_rf

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(min_samples_split,erreur_min_samples_split, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4);
plt.xlabel("Nombre minimum d'observations dans un noeud pour qu'une séparation soit effectuée")
plt.ylabel("Erreur")
plt.xticks(min_samples_split,min_samples_split); # Car le None ne peut pas être affiché

#### Résultat de la recherche

n_estimators = [10, 30, 50, 70, 100, 130, 160, 190, 220] :
- erreur = [0.8361054027017335, 0.7968092040149114, 0.7896074110110385, 0.7867732388450902, 0.7832596986819509, 0.7817079650265579, 0.7804837745630923,0.7802371972002211, 0.7797987830671329]

max_depth = [10,20, 30, 40, 50, 60, 70, 80, 90, 100, None] :
- erreur = [1.3591750963091456, 0.862434809822269, 0.8367067814091884, 0.836555349584275, 0.8361054027017335, 0.8361054027017335, 0.8361054027017335, 0.8361054027017335, 0.8361054027017335, 0.8361054027017335, 0.8361054027017335]

min_samples_split = [2, 5, 10, 20 ,30] :
- erreur = [0.8361054027017335, 0.835373537152044, 0.8412791214927986, 0.8654151778058987, 0.8917872715630332]


### Meilleur modèle


In [ ]:
best_model_rf = RandomForestRegressor(max_depth=30,n_estimators = 130, min_samples_split= 5,random_state=0,n_jobs=-1 )
best_model_rf.fit(X_train,y_train)
y_pred = best_model_rf.predict(X_test)
erreur = mean_squared_error(y_test, y_pred)**0.5
print('The rmsle of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
##############
#### Reset des forêts
##############

reset_rf_samples()


## Gradient Boosting avec LightGBM

In [ ]:
##############
#### Création des DataSets de LightGBM & des paramètres de ses datasets
##############

categorical_features = ["building_id", "site_id", "meter", "primary_use"]

X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(X_train, y_train, test_size=0.33, random_state=42)


lgb_train = lgb.Dataset(X_train_lgbm, 
                        label=y_train_lgbm,
                        categorical_feature=categorical_features,
                        free_raw_data=False)

lgb_eval = lgb.Dataset(X_test_lgbm, 
                       label=y_test_lgbm,
                       categorical_feature=categorical_features,
                       free_raw_data=False)




In [ ]:
##############
#### Paramètre du gradient boosting
##############
evals_result = {} 
params_lgb = {
    "objective": "regression",
    "boosting": "gbdt",
    "learning_rate": 0.1,
    "feature_fraction": 0.85,
    "metric": "rmse",
}


In [ ]:
##############
#### Entrainement du modele
##############

model_lgb = lgb.train(params_lgb, 
                train_set=lgb_train, 
                num_boost_round=1000, 
                valid_sets=[lgb_train,lgb_eval], 
                verbose_eval=50,
                evals_result=evals_result,
                early_stopping_rounds=50)

In [ ]:
y_pred=model_lgb.predict(X_test);
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
lgb.plot_importance(model_lgb, figsize=(10,10),grid=False)